In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/silver/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b7f88c0-5374-48d8-b115-2ad7245d90a2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 548ms :: artifacts dl 17ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [ ]:
# apagar tudo do hdfs

#to_path = "/projeto/fisico/bronze/bios/"
#to_path_aux = "/projeto/fisico/bronze/players_stats_by_season_full_details/"

#to_path_silver = "/projeto/fisico/silver/bios/"

#client.delete(to_path,True)
#client.delete(to_path_aux,True)

#client.delete(to_path_silver,True)

In [2]:
# declaracao dos caminhos

from_path = "../projeto/bronze/bios/"
from_path_aux = "../projeto/bronze/players_stats_by_season_full_details/"

hdfs_path = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/bios"
hdfs_path_aux = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/players_stats_by_season_full_details"

#hdfs_path1 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/bios1"
#hdfs_path_aux1 = "hdfs://hdfs-nn:9000/projeto/fisico/bronze/players_stats_by_season_full_details1"

In [ ]:
# upload de bronze

client.upload(to_path, from_path)
client.upload(to_path_aux, from_path_aux)

In [3]:
dataFrame = spark.read.csv(hdfs_path, header= True, inferSchema = True )
dataFrame_aux = spark.read.csv(hdfs_path_aux, header= True, inferSchema = True )

In [ ]:
dataFrame.printSchema()
dataFrame.show(truncate=False)
dataFrame.toPandas()
#topandas usar apenas para mostrar os dados em tabela

In [ ]:
dataFrame_status = dataFrame.summary() #sempre que se cria uma nova funçao é necessario um novo dataframe

dataFrame_status.toPandas() #serve para msotrar a informação de cima obtida

In [4]:
# JUNCAO

In [5]:
dataFrame_aux = dataFrame_aux.withColumn(    
    "year_start",
    dataFrame_aux.Season.substr(1, 4)
)

dataFrame_aux = dataFrame_aux.withColumn(    
    "year_end",
    dataFrame_aux.Season.substr(8, 4)
)

In [6]:
dataFrame_aux.createOrReplaceTempView("dataFrame_aux")

# preparar aux para a juncao.
df_aux = spark.sql(
    """
    SELECT Player,year_start,year_end,height,weight,birth_date,high_school
    FROM dataFrame_aux
    WHERE (Season == "2018 - 2019" OR Season == "2019 - 2020")
    AND League == "NBA"
    AND Stage == "Regular_Season"
    """
)

df_aux.show(truncate= False)

22/01/14 01:17:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------+----------+--------+------+------+------------+------------------------------------+
|Player               |year_start|year_end|height|weight|birth_date  |high_school                         |
+---------------------+----------+--------+------+------+------------+------------------------------------+
|James Harden         |2018      |2019    |6-5   |220   |Aug 26, 1989|Artesia High School                 |
|Paul George          |2018      |2019    |6-9   |220   |May 2, 1990 |Pete Knight High School             |
|Kemba Walker         |2018      |2019    |6-1   |184   |May 8, 1990 |Rice High School                    |
|Bradley Beal         |2018      |2019    |6-3   |207   |Jun 28, 1993|Chaminade College Preparatory School|
|Damian Lillard       |2018      |2019    |6-3   |195   |Jul 15, 1990|Oakland High School                 |
|Kevin Durant         |2018      |2019    |6-9   |240   |Sep 29, 1988|Montrose Christian School           |
|Giannis Antetokounmpo|2018 

In [7]:
df_aux = df_aux.withColumnRenamed('Player', 'name')

In [8]:
df_complete = dataFrame.unionByName(df_aux, allowMissingColumns = True)

df_complete.show(truncate= False)

+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+-----------+
|name               |year_start|year_end|position|height|weight|birth_date       |college                              |high_school|
+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+-----------+
|Alaa Abdelnaby     |1991      |1995    |F-C     |6-10  |240   |June 24, 1968    |Duke University                      |null       |
|Zaid Abdul-Aziz    |1969      |1978    |C-F     |6-9   |235   |April 7, 1946    |Iowa State University                |null       |
|Kareem Abdul-Jabbar|1970      |1989    |C       |7-2   |225   |April 16, 1947   |University of California, Los Angeles|null       |
|Mahmoud Abdul-Rauf |1991      |2001    |G       |6-1   |162   |March 9, 1969    |Louisiana State University           |null       |
|Tariq Abdul-Wahad  |1998      |2003    |F       |6-6   |223   |Novem

In [9]:
# INÍCIO DA EDIÇÃO DO DATAFRAME COMPLETO

In [10]:
# adicionar altura em inches para calculo do imc
replaced_df = df_complete.withColumn(    
    "height_in",
    (dataFrame.height.substr(1, 1) * 12) + dataFrame.height.substr(3, 2)
)

In [11]:
# adicionar imc
replaced_df = replaced_df.withColumn(    
    "imc",
    (col("weight") / (col("height_in")*col("height_in")) * 703)
)

In [12]:
# adicionar altura e peso em metric
replaced_df = replaced_df.withColumn(    
    "height_cm",
    col("height_in") * 2.54
)

replaced_df = replaced_df.withColumn(    
    "weight_kg",
    col("weight") * 0.45359237
)

In [13]:
# FIM DA EDIÇÃO DO DATAFRAME

In [14]:
replaced_df = replaced_df.withColumn("name", replaced_df["name"].cast(StringType())) 
replaced_df = replaced_df.withColumn("year_start", replaced_df["year_start"].cast(IntegerType()))
replaced_df = replaced_df.withColumn("year_end", replaced_df["year_end"].cast(IntegerType())) 
replaced_df = replaced_df.withColumn("position", replaced_df["position"].cast(StringType()))
replaced_df = replaced_df.withColumn("height", replaced_df["height"].cast(StringType())) 
replaced_df = replaced_df.withColumn("weight", replaced_df["weight"].cast(IntegerType())) 
replaced_df = replaced_df.withColumn("birth_date",replaced_df["birth_date"].cast(StringType())) 
replaced_df = replaced_df.withColumn("college", replaced_df["college"].cast(StringType())) 
replaced_df = replaced_df.withColumn("high_school", replaced_df["high_school"].cast(StringType()))
replaced_df = replaced_df.withColumn("height_in", replaced_df["height_in"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("imc", replaced_df["imc"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("height_cm", replaced_df["height_cm"].cast(DoubleType()))
replaced_df = replaced_df.withColumn("weight_kg", replaced_df["weight_kg"].cast(DoubleType()))

In [15]:
replaced_df.printSchema()
replaced_df.show(truncate=False)
replaced_df.toPandas()
#topandas usar apenas para mostrar os dados em tabela

root
 |-- name: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_end: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- college: string (nullable = true)
 |-- high_school: string (nullable = true)
 |-- height_in: double (nullable = true)
 |-- imc: double (nullable = true)
 |-- height_cm: double (nullable = true)
 |-- weight_kg: double (nullable = true)



+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+-----------+---------+------------------+------------------+------------------+
|name               |year_start|year_end|position|height|weight|birth_date       |college                              |high_school|height_in|imc               |height_cm         |weight_kg         |
+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+-----------+---------+------------------+------------------+------------------+
|Alaa Abdelnaby     |1991      |1995    |F-C     |6-10  |240   |June 24, 1968    |Duke University                      |null       |82.0     |25.09220701963117 |208.28            |108.8621688       |
|Zaid Abdul-Aziz    |1969      |1978    |C-F     |6-9   |235   |April 7, 1946    |Iowa State University                |null       |81.0     |25.179850632525532|205.74            |106.59420695      |


,name,year_start,year_end,position,height,weight,birth_date,college,high_school,height_in,imc,height_cm,weight_kg
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University,None,82.0,25.092207,208.28,108.862169
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University,None,81.0,25.179851,205.74,106.594207
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles",None,86.0,21.386560,218.44,102.058283
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University,None,73.0,21.370989,185.42,73.481964
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University,None,78.0,25.767423,198.12,101.151099
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,Matt Dellavedova,2019,2020,None,6-4,200.0,"Sep 8, 1990",None,Australian Institute of Sport,76.0,24.342105,193.04,90.718474
5118,Goga Bitadze,2019,2020,None,6-11,245.0,"Jul 20, 1999",None,None,83.0,25.001452,210.82,111.130131
5119,Nassir Little,2019,2020,None,6-6,220.0,"Feb 11, 2000",None,Orlando Christian Prep High School,78.0,25.420776,198.12,99.790321
5120,David Nwaba,2019,2020,None,6-4,219.0,"Jan 14, 1993",None,University City High School,76.0,26.654605,193.04,99.336729


In [15]:
replaced_df \
    .select("name","year_start","year_end","position","birth_date","college","height","height_in","height_cm","weight","weight_kg","imc") \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/silver/bios")

In [17]:
spark.sql(
    """
    SELECT *
    FROM silver.bios
    """
).show(truncate = False)

+-------------------+----------+--------+--------+-----------------+-------------------------------------+------+---------+------------------+------+------------------+------------------+
|name               |year_start|year_end|position|birth_date       |college                              |height|height_in|height_cm         |weight|weight_kg         |imc               |
+-------------------+----------+--------+--------+-----------------+-------------------------------------+------+---------+------------------+------+------------------+------------------+
|Alaa Abdelnaby     |1991      |1995    |F-C     |June 24, 1968    |Duke University                      |6-10  |82.0     |208.28            |240   |108.8621688       |25.09220701963117 |
|Zaid Abdul-Aziz    |1969      |1978    |C-F     |April 7, 1946    |Iowa State University                |6-9   |81.0     |205.74            |235   |106.59420695      |25.179850632525532|
|Kareem Abdul-Jabbar|1970      |1989    |C       |April 16, 